# Week 12 Examples: Blockchain & Multi-Party Attestation Systems

**Course:** Distributed Systems: Principles and Implementation  
**Week 12:** Blockchain & Multi-Party Attestation Systems  
**Topics:** Proof of Work, Multi-Party Attestations, Chain Validation

---

## Learning Objectives

By the end of this notebook, you will:
- Build a simple blockchain with Proof of Work (PoW)
- Understand how hash-based linking creates immutability
- Implement multi-party attestation chains
- Verify chain integrity across multiple parties

---

## Part 1: Simple Blockchain with Proof of Work

Let's build a minimal blockchain to understand the core concepts.

In [ ]:
import hashlib
import time
import json
from datetime import datetime

class Block:
    """A single block in the blockchain"""
    
    def __init__(self, index, data, previous_hash):
        self.index = index
        self.timestamp = time.time()
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        """Calculate SHA-256 hash of block contents"""
        block_string = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}{self.nonce}"
        return hashlib.sha256(block_string.encode()).hexdigest()
    
    def mine_block(self, difficulty):
        """Proof of Work: Find a nonce such that hash has 'difficulty' leading zeros"""
        target = "0" * difficulty
        
        print(f"Mining block {self.index}...")
        start_time = time.time()
        
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
        
        elapsed = time.time() - start_time
        print(f"Block mined! Hash: {self.hash}")
        print(f"Nonce: {self.nonce}, Time: {elapsed:.2f}s\n")
    
    def __repr__(self):
        return f"Block({self.index}, {self.data[:20]}..., {self.hash[:10]}...)"

# Test it out
print("=" * 60)
print("PART 1: Simple Blockchain with Proof of Work")
print("=" * 60 + "\n")

# Create genesis block (first block)
genesis = Block(0, "Genesis Block", "0")
genesis.mine_block(difficulty=4)  # 4 leading zeros

# Create second block
block1 = Block(1, "Alice sends 10 BTC to Bob", genesis.hash)
block1.mine_block(difficulty=4)

# Create third block
block2 = Block(2, "Bob sends 5 BTC to Charlie", block1.hash)
block2.mine_block(difficulty=4)

print("Blockchain created!")
print(f"Genesis: {genesis}")
print(f"Block 1: {block1}")
print(f"Block 2: {block2}")

### Why is this immutable?

Try changing the data in Block 1 and see what happens:

In [ ]:
print("\n" + "=" * 60)
print("Testing Immutability")
print("=" * 60 + "\n")

print(f"Original Block 1 hash: {block1.hash}")
print(f"Block 2 previous_hash: {block2.previous_hash}")
print(f"Match? {block1.hash == block2.previous_hash}\n")

# Tamper with Block 1
print("Tampering with Block 1 data...")
block1.data = "Alice sends 100 BTC to Bob"  # Changed from 10 to 100!
block1.hash = block1.calculate_hash()  # Recalculate hash

print(f"New Block 1 hash:      {block1.hash}")
print(f"Block 2 previous_hash: {block2.previous_hash}")
print(f"Match? {block1.hash == block2.previous_hash}\n")

print("❌ Chain is broken! Block 2's previous_hash no longer matches Block 1's hash.")
print("This is how blockchains detect tampering.")

---

## Part 2: Multi-Party Attestation System

Now let's build a simplified version of the AdChain attestation system. Multiple parties create cryptographically-signed attestations that reference each other.

In [ ]:
import uuid

class Attestation:
    """A cryptographically-signed attestation from a party"""
    
    def __init__(self, party_name, event_type, event_data, correlation_id, previous_cid=None):
        self.party_name = party_name
        self.event_type = event_type
        self.event_data = event_data
        self.correlation_id = correlation_id  # Links related attestations
        self.previous_cid = previous_cid  # Reference to previous attestation (like blockchain)
        self.timestamp = time.time()
        self.cid = self.calculate_cid()  # Content Identifier (hash)
    
    def calculate_cid(self):
        """Calculate Content Identifier (CID) - hash of attestation contents"""
        attestation_string = json.dumps({
            "party": self.party_name,
            "type": self.event_type,
            "data": self.event_data,
            "correlation_id": self.correlation_id,
            "previous_cid": self.previous_cid,
            "timestamp": self.timestamp
        }, sort_keys=True)
        return hashlib.sha256(attestation_string.encode()).hexdigest()[:16]  # Shortened for readability
    
    def __repr__(self):
        return f"Attestation({self.party_name}, {self.event_type}, CID: {self.cid})"

# Simulate ad delivery with multi-party attestations
print("\n" + "=" * 60)
print("PART 2: Multi-Party Attestation System")
print("=" * 60 + "\n")

# Generate a unique correlation ID for this ad delivery event
correlation_id = str(uuid.uuid4())[:8]
print(f"Correlation ID: {correlation_id}")
print(f"Event: Ad delivery for user 'alice123'\n")

# Party 1: Publisher attests they served content
publisher_attestation = Attestation(
    party_name="Publisher (gaming.example)",
    event_type="content_served",
    event_data={"user": "alice123", "content": "gaming_article_42"},
    correlation_id=correlation_id,
    previous_cid=None  # First in chain
)
print(f"✅ {publisher_attestation}")
print(f"   Data: {publisher_attestation.event_data}")
print(f"   Timestamp: {datetime.fromtimestamp(publisher_attestation.timestamp)}\n")

# Party 2: Ad Network attests they delivered an ad
ad_network_attestation = Attestation(
    party_name="Ad Network",
    event_type="ad_delivered",
    event_data={"user": "alice123", "ad_id": "sneakers_ad_789"},
    correlation_id=correlation_id,
    previous_cid=publisher_attestation.cid  # References publisher's attestation
)
print(f"✅ {ad_network_attestation}")
print(f"   Data: {ad_network_attestation.event_data}")
print(f"   References: {ad_network_attestation.previous_cid}\n")

# Party 3: Browser Monitor attests the ad was actually rendered
browser_monitor_attestation = Attestation(
    party_name="Browser Monitor",
    event_type="ad_rendered",
    event_data={"user": "alice123", "ad_id": "sneakers_ad_789", "visible": True},
    correlation_id=correlation_id,
    previous_cid=ad_network_attestation.cid  # References ad network's attestation
)
print(f"✅ {browser_monitor_attestation}")
print(f"   Data: {browser_monitor_attestation.event_data}")
print(f"   References: {browser_monitor_attestation.previous_cid}\n")

### Chain Validation

Now let's verify the attestation chain is valid:

In [ ]:
def validate_attestation_chain(attestations):
    """Verify that all attestations in the chain are valid"""
    print("\n" + "=" * 60)
    print("Validating Attestation Chain")
    print("=" * 60 + "\n")
    
    # Check 1: All attestations have the same correlation_id
    correlation_ids = set(a.correlation_id for a in attestations)
    if len(correlation_ids) != 1:
        print("❌ INVALID: Multiple correlation IDs found")
        return False
    print(f"✅ All attestations share correlation_id: {list(correlation_ids)[0]}")
    
    # Check 2: Chain links are valid (each previous_cid matches the previous attestation's CID)
    for i in range(1, len(attestations)):
        expected_cid = attestations[i-1].cid
        actual_cid = attestations[i].previous_cid
        
        if expected_cid != actual_cid:
            print(f"❌ INVALID: Chain broken at attestation {i}")
            print(f"   Expected previous_cid: {expected_cid}")
            print(f"   Actual previous_cid:   {actual_cid}")
            return False
    
    print(f"✅ All {len(attestations)} attestations are properly linked")
    
    # Check 3: Timestamps are in order
    for i in range(1, len(attestations)):
        if attestations[i].timestamp < attestations[i-1].timestamp:
            print(f"❌ INVALID: Timestamp out of order at attestation {i}")
            return False
    
    print(f"✅ Timestamps are in chronological order")
    
    print("\n🎉 CHAIN IS VALID! All attestations verified.")
    return True

# Validate the chain
chain = [publisher_attestation, ad_network_attestation, browser_monitor_attestation]
validate_attestation_chain(chain)

### What if someone lies?

Let's simulate a malicious ad network that claims to deliver a different ad:

In [ ]:
# Malicious ad network claims they delivered a different ad
malicious_ad_network = Attestation(
    party_name="Ad Network (MALICIOUS)",
    event_type="ad_delivered",
    event_data={"user": "alice123", "ad_id": "FAKE_AD_999"},  # Different ad!
    correlation_id=correlation_id,
    previous_cid=publisher_attestation.cid
)

# Browser monitor still attests the real ad
# But now it references the malicious attestation
browser_monitor_attestation_v2 = Attestation(
    party_name="Browser Monitor",
    event_type="ad_rendered",
    event_data={"user": "alice123", "ad_id": "sneakers_ad_789", "visible": True},  # Real ad
    correlation_id=correlation_id,
    previous_cid=malicious_ad_network.cid
)

print("\n" + "=" * 60)
print("Detecting Malicious Behavior")
print("=" * 60 + "\n")

print(f"Ad Network claims: {malicious_ad_network.event_data['ad_id']}")
print(f"Browser Monitor saw: {browser_monitor_attestation_v2.event_data['ad_id']}\n")

# Auditor detects mismatch
if malicious_ad_network.event_data['ad_id'] != browser_monitor_attestation_v2.event_data['ad_id']:
    print("❌ FRAUD DETECTED!")
    print("   Ad Network and Browser Monitor disagree on which ad was shown.")
    print("   This is Byzantine fault tolerance in action!")
else:
    print("✅ All parties agree")

---

## Summary

**What we learned:**

1. **Blockchain basics:** Hash-based linking creates immutability
2. **Proof of Work:** Computational puzzles for leader election (expensive!)
3. **Multi-party attestations:** Multiple independent parties create signed records
4. **Chain validation:** Verify all attestations match and are properly linked
5. **Byzantine fault tolerance:** System detects when parties disagree

**For your final project:**

You'll build a similar system for a domain of your choice (ad delivery, credentials, supply chain, etc.) with:
- 3+ parties creating attestations
- Cryptographic signatures
- Chain validation
- Event correlation
- Auditor verification

**Next:** Check out the homework notebook to build your own attestation system!